In [1]:
from bs4 import BeautifulSoup
import requests
from db import PythonProblems
from datetime import datetime
from urllib.parse import urljoin
from tqdm import tqdm

### Creating and connecting to database

In [2]:
db = PythonProblems('python.sqlite', 'init_tables.sql')

### Getting the pages to be crawled

In [7]:
%%time
urls = [
       # "https://www.w3resource.com/python-exercises/python-basic-exercises.php", 
       # "https://www.w3resource.com/python-exercises/basic/", # commit d66ecae4b3ee871ca6d94d90ac9fde7d28dd1c67
       # "https://www.w3resource.com/python-exercises/string/",
       # "https://www.w3resource.com/python-exercises/list/", # commit 42e8eb9a78d7474cecf2c373e3c9c9b4652423e7
       # "https://www.w3resource.com/python-exercises/dictionary/", # commit 6490b5779a9f472edbe7cd286d579557b2b53411
       # "https://www.w3resource.com/python-exercises/tuple/", # commit aa04882c57893437f896b46e755433a64b34c1c6
       # "https://www.w3resource.com/python-exercises/sets/", # commit ebd13e285ccaa3437b6b7076d01b18d5f9c06277
       # "https://www.w3resource.com/python-exercises/array/", # commit 5d43e87ca2614bc42b11a3ada50616c48eac5dd8
        "https://www.w3resource.com/python-exercises/python-conditional-statements-and-loop-exercises.php"]
pms = []
for idx, url in enumerate(urls):
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html5lib')
    solution_links, problem_comparison, problems = get_problems(soup)
    
    #if idx == 0:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[145]["content"] = "Write a Python program to test if a variable is a list or tuple or a set."
    
    #if idx == 5:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[21]["content"] = "Write a Python program to remove an empty tuple(s) from a list of tuples."
    
    #if idx == 8:
    if idx == 0:
        # Bug-fix, problem is written wrong in the first page
        problems[25]['content'] = "Write a Python program to print alphabet pattern 'S'"
        problems.insert(26, problems[26].copy())
        problems[26]['content'] = "Write a Python program to print alphabet pattern 'T'"
        solution_links = []
        aTag = soup.find_all('a')
        for a in aTag:
            if a.text.strip() == "Click me to see the sample solution":
                solution_links.append(a)
        problem_comparison.insert(26, "")
        
        problems[33]['content'] = "Write a Python program to sum of two given integers. However, if the sum is between 15 to 20 it will return 20."
    
    # Dictionary has problem repetition, but different solutions
    if idx == 4:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison, problems)
    else:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison)
        
    pms.append(potential_mismatch)
    insert(problems, solutions)
#db.close_connection()

100%|██████████| 45/45 [00:00<00:00, 7136.14it/s]
44it [01:03,  1.41s/it]

[{'problem': {'title': 'Python conditional statements and loops', 'link': 'https://www.w3resource.com/python-exercises/python-conditional-statements-and-loop-exercises.php', 'content': 'Write a Python program to find those numbers which are divisible by 7 and multiple of 5, between 1500 and 2700 (both included).', 'retrieved_date': datetime.datetime(2017, 12, 11, 23, 58, 56, 928592), 'crawler': 'W3Resource'}, 'solution': {'content': "nl=[]\nfor x in range(1500, 2700):\n    if (x%7==0) and (x%5==0):\n        nl.append(str(x))\nprint (','.join(nl))\n", 'link': 'https://www.w3resource.com/python-exercises/python-conditional-exercise-1.php', 'retrieved_date': datetime.datetime(2017, 12, 11, 23, 58, 57, 787061), 'ignore': False, 'idx': 0}}, {'problem': {'title': 'Python conditional statements and loops', 'link': 'https://www.w3resource.com/python-exercises/python-conditional-statements-and-loop-exercises.php', 'content': 'Write a Python program to convert temperatures to and from celsius, f

IndexError: list index out of range

## Functions

### Problems and solutions links

In [3]:
def get_problems(soup):
    exercises = soup.find_all("strong")
    problem_list = []
    solution_links = []
    problem_comparison = []
    
    # First replace <br> with line break
    for br in soup.find_all("br"):
        br.replace_with("\n")

    title = soup.find_all("h2", class_="heading")[0].text.split("[")[0].strip()

    for e in tqdm(exercises):
        # Find solutions links
        solution_link = e.findNext("a")
        try:
            while solution_link.text.strip() != "Click me to see the sample solution":
                solution_link = solution_link.findNext("a")
        except AttributeError:
            continue
        solution_links.append(solution_link)

        # Get problem
        problem_text = ""
        problem_text = e.parent.text
        nextTag = e.parent.next_sibling
        while not isinstance(nextTag, type(e)):
            nextTag = nextTag.next_sibling
        if nextTag and nextTag.name == "pre":
            problem_text += nextTag.text

        # Get original text for comparison in solution's page
        editor_partition = problem_text.partition("Go to the editor")
        #Go to the editor is written
        if editor_partition[2] != '':
            problem_comparison.append(editor_partition[0].split('.')[1].strip())
        # Go to the editor is not written
        else:
            problem_comparison.append(problem_text.split('.')[1])

        problem_text = problem_text.replace("Go to the editor", "")
        problem_text = problem_text.replace("Click me to see the sample solution", "")
        problem_text = problem_text.partition('.')[2].strip()
        problem_list.append(problem_text)

    problems = []

    for problem_text in problem_list:
        problems.append({"title": title, "link": url,
                         "content": problem_text, "retrieved_date": datetime.now(),
                         "crawler": "W3Resource"})
        
    return solution_links, problem_comparison, problems

### Solutions

In [4]:
def get_solutions(solution_links, problem_comparison, problems=None):
    solutions = []
    potential_mismatch = []
    
    if problems:
        problems_list = [p['content'] for p in problems]
    
    for idx, link in tqdm(enumerate(solution_links)):
        link_sol = urljoin(url, link['href'])
        response = requests.get(link_sol)
        soup = BeautifulSoup(response.text, 'html5lib')

        # Compare problem inside with original problem
        compare_problem = soup.find_all("h2")[0].findNext().text
        if compare_problem[:-1].strip() != problem_comparison[idx].strip():
            potential_mismatch.append(link_sol + '\n' + compare_problem[:-1].strip() + '\n' + problem_comparison[idx] + '\n\n')

        code = soup.find_all("code")
        
        if problems:
            idx_sol = problems_list.index(problems_list[idx])
        else:
            idx_sol = idx
        solutions.append({"content": code[0].text, "link": link_sol,
                          "retrieved_date": datetime.now(), "ignore": False,
                          "idx": idx_sol})
    return solutions, potential_mismatch

### Insert rows

In [5]:
def insert(problems, solutions):
    db.insert_rows(problems, solutions)
    tp, ts, rp, rs = db.populate()
    print("Got %d problems inserted, %d solutions inserted, %d problems repeated and %d solutions repeated" % (
          tp, ts, rp, rs))

### Control check

In [50]:
for item in pms[0]:
    print(item)

https://www.w3resource.com/python-exercises/python-conditional-exercise-10.php
Write a Python program which iterates the integers from 1 to 50. For multiples of three print "Fizz" instead of the number and for the multiples of five print "Buzz". For numbers which are multiples of both three and five print "FizzBuzz".
 Write a Python program which iterates the integers from 1 to 50


https://www.w3resource.com/python-exercises/python-conditional-exercise-11.php
Write a Python program which takes two digits m (row) and n (column) as input and generates a two-dimensional array. The element value in the i-th row and j-th column of the array should be i*j.
Note :
i = 0,1.., m-1 
j = 0,1, n-1
Write a Python program which takes two digits m (row) and n (column) as input and generates a two-dimensional array


https://www.w3resource.com/python-exercises/python-conditional-exercise-15.php
Write a Python program to check the validity of a password (input from users).

Validation :
Write a Python

In [10]:
solutions

[{'content': "nl=[]\nfor x in range(1500, 2700):\n    if (x%7==0) and (x%5==0):\n        nl.append(str(x))\nprint (','.join(nl))\n",
  'idx': 0,
  'ignore': False,
  'link': 'https://www.w3resource.com/python-exercises/python-conditional-exercise-1.php',
  'retrieved_date': datetime.datetime(2017, 12, 11, 23, 47, 19, 298600)},
 {'content': 'temp = input("Input the  temperature you like to convert? (e.g., 45F, 102C etc.) : ")\ndegree = int(temp[:-1])\ni_convention = temp[-1]\n\nif i_convention.upper() == "C":\n  result = int(round((9 * degree) / 5 + 32))\n  o_convention = "Fahrenheit"\nelif i_convention.upper() == "F":\n  result = int(round((degree - 32) * 5 / 9))\n  o_convention = "Celsius"\nelse:\n  print("Input proper convention.")\n  quit()\nprint("The temperature in", o_convention, "is", result, "degrees.")\n',
  'idx': 1,
  'ignore': False,
  'link': 'https://www.w3resource.com/python-exercises/python-conditional-exercise-2.php',
  'retrieved_date': datetime.datetime(2017, 12, 11,